In [16]:
from datasets import load_dataset, concatenate_datasets, Dataset
from copy import deepcopy

In [4]:
ds = load_dataset("gaotang/sky_v02_filtered_2_5kcode_18kmath_math_code_sky")


In [21]:
ds

DatasetDict({
    train: Dataset({
        features: ['context_messages', 'winner'],
        num_rows: 75414
    })
})

In [ ]:
ds['train'][0]

{'context_messages': [{'content': "Please act as an impartial judge and evaluate the quality of the responses provided by two AI chatbots to the client question displayed below. Begin your evaluation by first generating the rubric items. Enclose this section within <rubric> and </rubric> tags. Then, compare the following two conversations between the client and the AI chatbots, and provide your evaluation according to the rubric items. Ensure that the order in which the responses are presented does not influence your decision, and do not let response length or chatbot names affect your evaluation. Be as objective as possible. Enclose your complete evaluation explanation and final verdict within <eval> and </eval> tags. After providing your explanation, output your final verdict by strictly following this format: '[[A]]' if Chatbot A is better, '[[B]]' if Chatbot B is better.i.e., <rubric>rubric items here</rubric>\n\n<eval>detailed evaluation here according to the rubric items</eval>\n

In [13]:
SFT_SYSTEM =( "Please act as an impartial judge and evaluate the quality of the responses provided by two AI chatbots. "
            "You should choose the chatbot that follows the client's instructions and answers the client's question better. "
            "Do not allow the length of the responses to influence your evaluation. Do not favor certain names "
            "of the chatbots. Be as objective as possible. Output your final verdict directly by strictly following this format: "
            '"A" if Chatbot A is better, "B" if Chatbot B is better.')

In [14]:
SFT_SYSTEM

'Please act as an impartial judge and evaluate the quality of the responses provided by two AI chatbots. You should choose the chatbot that follows the client\'s instructions and answers the client\'s question better. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the chatbots. Be as objective as possible. Output your final verdict directly by strictly following this format: "A" if Chatbot A is better, "B" if Chatbot B is better.'

In [22]:
sft_ds = []

content_messages = []
winner = []
for item in ds['train']:
    copy_item = deepcopy(item)
    assert copy_item['context_messages'][0]['role'] == "system", "Not system first"
    copy_item['context_messages'][0]['content'] = SFT_SYSTEM 
    if copy_item["winner"] == "model_a":
        copy_item['winner'] = "A" 
    elif copy_item['winner'] == "model_b":
        copy_item['winner'] = "B"
    else:
        raise NotImplementedError("Check your winner")  
    content_messages.append(copy_item['context_messages'])
    winner.append(copy_item['winner'])
    sft_ds.append(copy_item)

In [23]:

sft_dataset = Dataset.from_dict({
        'context_messages': content_messages,
        'winner': winner
    })

In [24]:
sft_dataset

Dataset({
    features: ['context_messages', 'winner'],
    num_rows: 75414
})

In [25]:
sft_dataset[0]

{'context_messages': [{'content': 'Please act as an impartial judge and evaluate the quality of the responses provided by two AI chatbots. You should choose the chatbot that follows the client\'s instructions and answers the client\'s question better. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the chatbots. Be as objective as possible. Output your final verdict directly by strictly following this format: "A" if Chatbot A is better, "B" if Chatbot B is better.',
   'role': 'system'},
  {'content': '[Client Question]\nI was reading up on the Fibonacci Sequence,  $1,1,2,3,5,8,13,\\ldots $  when I noticed some were able to calculate specific numbers. So far I\'ve only figured out creating an array and counting to the value, which is incredibly simple, but I reckon I can\'t find any formula for calculating a Fibonacci number based on it\'s position. \n Is there a way to do this? If so, how are we able to apply these formulas to array

In [26]:
sft_dataset.push_to_hub("gaotang/sky_v02_filtered_2_5kcode_18kmath_math_code_sky_sft")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/76 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/gaotang/sky_v02_filtered_2_5kcode_18kmath_math_code_sky_sft/commit/a2603228dd4bc84ec9ef2685abbb716ea227d8c3', commit_message='Upload dataset', commit_description='', oid='a2603228dd4bc84ec9ef2685abbb716ea227d8c3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/gaotang/sky_v02_filtered_2_5kcode_18kmath_math_code_sky_sft', endpoint='https://huggingface.co', repo_type='dataset', repo_id='gaotang/sky_v02_filtered_2_5kcode_18kmath_math_code_sky_sft'), pr_revision=None, pr_num=None)